## intro
Docker is an integral piece to registering a Task on GBDX. In previous tutorials, we've demonstrated how to build a Docker image locally from a Dockerfile and then push that Docker image to Docker Hub to be used in the registration process. To test that the Task works as expected, we've simply used that Task in a Workflow, and then checked the status of the Workflow. If the Workflow succeeded, we could assume that we wrote and registered the Task successfully. 

If you'd like to debug the Task code outside of the Workflow system, it's easy enough to test the Task Docker locally. You can simply mount test data in your Docker container in a way that mimics the way GBDX mounts the input data to the Docker container when it's running your Task within a Workflow. You can then run the Task code in the local Docker container and debug from there. This is a faster way to debug your code, and it's one that doesn't use AWS resources. 

__Pre-requisites:__
This tutorial will build off of the string-ports-tutorial.ipynb, where we built the 'Doughnut Task'. We'll re-use the same Task code and Dockerfile that we built during that tutorial.

## 1. Get test data
To actively participate in this tutorial, you will need to get your hands on some test data. Specifically, you will need a raster and a shapefile that will clip that raster. You can use your own test data, or we describe how to get test data below. We'll also need a ports.json file, but we'll write that as part of the tutorial. 

__raster:__ if you've completed the gbdxtools-tutorial.ipynb, you will likely have run the AOP task on the image with the Cat ID '10400100245B7800', and it will be stored in your customer S3 bucket under 'demo_output/aop_10400100245B7800/'. You can download this image directly from the [S3 browser](http://s3browser.geobigdata.io/login.html). 

__shapefile:__ download the shapefile 'this_shp_will_clip_10400100245B7800.zip' from the [github repo](https://github.com/GeoBigData/gbdx-training/tree/master/advanced_concepts) where you downloaded this notebook, and un-zip it. 

___
## 2. Write a test ports.json file
We want to mimic what GBDX will do when it spins up your Task, which is to mount input data and a ports.json file to the running Docker container. If we were to call the Doughnut Task in gbdxtools and specify the clip selection like so...

```python
doughnut_task = gbdx.Task("doughnut_clip", input_raster='<path to raster>', input_shapefile='<path to shapefile>', clip_selection="doughnut")
```

...GBDX will generate a ports.json file like so and mount it to the running Docker container:

```json
{
    "clip_selection":"doughnut"
}
```

To test the Task code within the Task Docker, we can write a ports.json file with the same contents. 

#### 2.1 Navigate to the folder with your test data by filling in the full path to the data directory in the command below and executing this cell. 

In [ ]:
cd <full-path-to-test-data-directory>

#### 2.2 Write a ports.json file and add the example json code just described. Or run the code in the following cell, which will write the file for you. 

In [ ]:
%%writefile ports.json
{
    "clip_selection":"doughnut"
}

___
## 3. Mount test data to a Docker container
In this next step, we will run a Docker container from the Docker image that we built in the previous string-ports.ipynb tutorial notebook. When we start the container, we'll add the Docker command to mount a data volume so that we can test the Task code within the container. 

#### 3.1 Start Docker and bring up a terminal window

#### 3.2 Run the following Docker command to start a container with mounted test data, filling in the full path to the test raster, shapefile, and ports.json files saved on your computer, and the name of the doughnut docker image that you previously built

Linux:

```
docker run -it --rm -v <local-path-to-raster>:/mnt/work/input/input_raster -v <local-path-to-shapefile >:/mnt/work/input/input_shapefile -v <local-path-to-ports.json>:/mnt/work/input/ports.json <docker-hub-username/repository-name> bash
```
Windows:

```
docker run -it --rm -v c:\<local-path-to-raster>:c:\mnt\work\input_raster -v c:\<local-path-to-shapefile >:c:\mnt\work\input_shapefile -v c:\<local-path-to-ports.json>:c:\mnt\work\input\ports.json <docker-hub-username/repository-name> bash
```

For example, my Docker command looks like this:

```
docker run -it --rm -v /Users/elizabethgolden/Documents/test_data/input_raster:/mnt/work/input/input_raster -v /Users/elizabethgolden/Documents/test_data/input_shapefile:/mnt/work/input/input_shapefile -v /Users/elizabethgolden/Documents/test_data/ports.json:/mnt/work/input/ports.json gbdxtrainer/doughnut_docker bash
```

Here's a breakdown of the above Docker run command:

>`docker` : the base command for the Docker CLI
>
>`run` : run the following commands in a new container
>
>`-it` : interactive mode
>
>`--rm` : removes container upon exit
>
>`-v` : bind mount a volume
>
>`<path-to-test-raster>` : this will be the absolute filepath to the local directory where you have saved a test raster/shapefile/port.json 
>
>`/mnt/work/input/input_raster` : the directory where the test raster/shapefile/ports.json you specified will be copied to in the Docker container
>
>`<username/repository-name>` : the name of your tagged Docker image, which will likely be in the format of your Docker Hub username and the name of your repository on Docker Hub for this image
>
>`bash` : run container with bash prompt 


#### 3.3 You can now nagivate within the Docker container to the mounted test data using the following command:

```
cd mnt/work/input
```

#### 3.4 You should see your test data when you list the contents of the input directory

```
ls
```

For example: 

```
root@1539752ff9b6:/# cd mnt/work/input/
root@1539752ff9b6:/mnt/work/input# ls
input_raster  input_shapefile  ports.json
```

___
## 4. Test the Task code
You're all set to test the code within the Docker container.  

#### 4.1 Navigate back to the root directory, then navigate to the doughnut_task.py script using the following commands:

```
cd /my_scripts/
```

#### 4.2 Then execute the doughnut_task.py script

```
python doughnut_task.py
```

#### 4.3 When the script completes, you should be able to back out to the root directory, then navigate to the output directory to see the output of the doughnut_task.py using the following commands:

```
cd /mnt/work/output/
```

## conclusion
You've completed the basic steps for testing Task code within a running Docker container by mounting test data volumes. Testing your Task code in the Docker container simplifies the debugging process and is a recommended step before pushing the Docker to Docker Hub. 